# Image Creation using PixelSorter

This Jupyter Notebook demonstrates the process of creating images using the `PixelSorter` library. The workflow involves several steps, including image loading, segmentation, flow field application, and pixel sorting. Below is a brief overview of the steps involved:



1. **Import Necessary Libraries**: Import all the required libraries such as `Mask`, `PixelSorter`, `FlowField`, `ColorSpace`, `cv2`, `numpy`, and `os`.


In [1]:
from mask import Mask
from pixelsorter import PixelSorter
from flowfield import FlowField
from colorspace import ColorSpace
import cv2
import numpy as np
import os


c:\Users\User\.conda\envs\i2dl\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



2. **Define Helper Functions**: Define any helper functions needed, such as `show_image` to display images.


In [2]:
def show_image(img):
    cv2.imshow('Image', img)
    cv2.resizeWindow('Image', 600,600)
    cv2.waitKey(0)
    cv2.destroyAllWindows()


3. **Load and Preprocess Image**: Load the input image, resize it, and display it. Here are the Configurations listed


In [3]:
    pictures_name = 'statue.png'
    use_segmentation = True
    use_flowfield = True
    flowfield_type = 'depth'



In [4]:

    image = cv2.imread(os.path.join('pictures','input', pictures_name))
    image = cv2.resize(image, (image.shape[1]//2, image.shape[0]//2))
    show_image(image)

    pixelSorter = PixelSorter()
    mask = Mask()



4. **Segmentation and Mask Application**: Depending on the configuration, apply segmentation masks to the image.


In [5]:

    if use_segmentation:
        applied_masks = mask.segmentate

    else:
        applied_masks = [image]


AttributeError: 'Mask' object has no attribute 'segmentate'


5. **Flow Field Application**: If enabled, apply a flow field to the image based on a specified type (e.g., depth).


In [6]:

    if use_flowfield:
        image_depth = ColorSpace.get_sort_func(flowfield_type)(image) 
        flow_field = FlowField(image_depth)
        show_image(image_depth)



6. **Pixel Sorting**: Use the `PixelSorter` to sort the pixels of the image based on various criteria such as hue, value, and flow field direction.


In [7]:


    cummulativ_image = np.zeros_like(image)
    for applied_seg_image in applied_masks:
        masked_image,reverse = mask.mask_image(applied_seg_image, 'saturation', threshold=(175,255))
        masked_image,second_reverse = mask.mask_image(mask.apply_mask(image, masked_image), 'value', threshold=(105,195))
 
        show_image(mask.apply_mask(image, masked_image))    

        sorted_image = pixelSorter.sort_pixels(image,
                                                mask=masked_image, 
                                                sort_type='hue', 
                                                sort_direction='vertical', 
                                                sort_ascend=True,
                                                split_interval_groups = True,
                                                interval_random_range = (image.shape[0],image.shape[0]),
                                                flow_field = None)
        sorted_image = pixelSorter.sort_pixels(sorted_image,
                                                mask=masked_image, 
                                                sort_type='value', 
                                                sort_direction='flowfield', 
                                                sort_ascend=True,
                                                split_interval_groups = True,
                                                interval_random_range = (image.shape[1],image.shape[1]),
                                                flow_field = flow_field)
        
        output_image = sorted_image + mask.apply_mask(image, np.clip(reverse + second_reverse,0,1)) 
        cummulativ_image += output_image
        show_image(output_image)



The resulting image will be displayed at the end of the notebook.

In [8]:

    show_image(cummulativ_image)
     #cv2.imwrite(os.path.join('pictures','output', pictures_name), img)